In [1]:
#Initialize and load weather dataframe

from pyspark import SparkContext
sc = SparkContext(master="local[4]")
#sc.version

import os
import sys

from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType
%pylab inline

# Just like using Spark requires having a SparkContext, using SQL requires an SQLContext
sqlContext = SQLContext(sc)

Populating the interactive namespace from numpy and matplotlib


In [2]:
from os.path import split,join,exists
from os import mkdir,getcwd,remove
from glob import glob

# create directory if needed


data_dir='../../Data'
weather_dir=join(data_dir,'Weather')
!ls $weather_dir

NY.parquet NY.tgz


In [3]:
if exists(weather_dir):
    print('directory',weather_dir,'already exists')
else:
    print('making',weather_dir)
    mkdir(weather_dir)

directory ../../Data/Weather already exists


In [4]:
file_index='NY'
zip_file='%s.tgz'%(file_index) #the .csv extension is a mistake, this is a pickle file, not a csv file.
old_files='%s/%s*'%(weather_dir,zip_file[:-3])
for f in glob(old_files):
    print('removing',f)
    !rm -rf {f}

command="curl https://mas-dse-open.s3.amazonaws.com/Weather/by_state/%s > %s/%s "%(zip_file, weather_dir,zip_file)
print(command)
!$command
!ls -lh $weather_dir/$zip_file

removing ../../Data/Weather/NY.tgz
removing ../../Data/Weather/NY.parquet
curl https://mas-dse-open.s3.amazonaws.com/Weather/by_state/NY.tgz > ../../Data/Weather/NY.tgz 
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22.1M  100 22.1M    0     0  12.2M      0  0:00:01  0:00:01 --:--:-- 12.2M
-rw-r--r--  1 yoavfreund  staff    22M Apr 12 16:15 ../../Data/Weather/NY.tgz


In [5]:
#extracting the parquet file
!tar zxvf {weather_dir}/{zip_file} -C {weather_dir}

x NY.parquet/
x NY.parquet/_SUCCESS
x NY.parquet/part-00000-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.parquet
x NY.parquet/part-00001-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.parquet
x NY.parquet/part-00002-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.parquet
x NY.parquet/part-00003-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.parquet
x NY.parquet/part-00004-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.parquet
x NY.parquet/part-00005-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.parquet
x NY.parquet/part-00006-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.parquet
x NY.parquet/part-00007-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.parquet
x NY.parquet/part-00008-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.parquet
x NY.parquet/part-00009-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.parquet
x NY.parquet/part-00010-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.parquet
x NY.parquet/part-00011-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.

In [6]:
weather_parquet = join(weather_dir,zip_file[:-3]+'parquet')
print(weather_parquet)
df = sqlContext.read.load(weather_parquet)
df.show(1)

../../Data/Weather/NY.parquet
+-----------+-----------+----+--------------------+-----+
|    Station|Measurement|Year|              Values|State|
+-----------+-----------+----+--------------------+-----+
|USC00303452|       PRCP|1903|[00 7E 00 7E 00 7...|   NY|
+-----------+-----------+----+--------------------+-----+
only showing top 1 row



## Comparing GroupBy and ReduceByKey

In [7]:
def Use_Iterator(X):
    key,iter=X
    s=0
    for pair in iter:
        s+= pair[1]
    return (key,s)

In [13]:
%%time
A=df.select('Station','Year').rdd.map(lambda row: (row['Station'],row['Year']))

CPU times: user 1.81 ms, sys: 1.84 ms, total: 3.65 ms
Wall time: 41.5 ms


In [14]:
A.take(3)

[('USC00303452', 1903), ('USC00303452', 1904), ('USC00303452', 1905)]

In [15]:
AA=A
for i in range(100):
    AA=AA.union(A)
print(A.count(),AA.count())

84199 8504099


In [17]:
AA.getNumPartitions(),A.getNumPartitions()

(404, 4)

In [30]:
AA=AA.repartition(10).cache()

In [31]:
AA.getNumPartitions(),A.getNumPartitions()

(10, 4)

In [32]:
B=AA.groupBy(lambda pair:pair[0]).map(Use_Iterator)
B.take(3)

[('USC00307524', 21324534),
 ('USC00308248', 98745882),
 ('USC00302610', 138988221)]

In [ ]:
C=B.collect()
C[:4]

In [33]:
%%time
g=AA.reduceByKey(lambda x,y:x+y)

CPU times: user 5.19 ms, sys: 2.28 ms, total: 7.47 ms
Wall time: 13.8 ms


In [34]:
%%time
C2=g.collect()

CPU times: user 5.78 ms, sys: 3.22 ms, total: 9 ms
Wall time: 2.44 s


In [ ]:
C2[:4]

In [ ]:
!pip install pyarrow

In [ ]:
from pyspark.sql.functions import pandas_udf, PandasUDFType

In [ ]:
@pandas_udf("Station string, Measurement string, Year int, State string", PandasUDFType.GROUPED_MAP)
def substract_mean(pdf):
    # pdf is a pandas.DataFrame
    return len(pdf.columns)

In [ ]:
df.drop('Values').groupby("Station").apply(substract_mean).show(1)

In [ ]:
df.schema

In [ ]:
df.show(1)

In [ ]:
"Station string, Measurement string, Year int, Values binary, State string"